In [25]:
#read in libraries
import os
import json
import random
import requests

from ipyleaflet import Map, GeoJSON, LayersControl, LegendControl, ScaleControl

import ipyleaflet
import json
import pandas as pd
import os
import requests
from ipywidgets import link, FloatSlider, HTML
from branca.colormap import linear
from ipyleaflet import Choropleth, Map, Popup

In [12]:
#read in statistical areas (geographic regions)
if not os.path.exists('Neighborhood_Statistical_Areas.geojson'):
    url = 'https://opendata.arcgis.com/datasets/e7daa4c977d14e1b9e2fa4d7aff81e59_0.geojson'
    r = requests.get(url)
    with open('Neighborhood_Statistical_Areas.geojson', 'w') as f:
        f.write(r.content.decode("utf-8"))

#load in geojson data       
with open('Neighborhood_Statistical_Areas.geojson', 'r') as f:
    data = json.load(f)
    
#another way
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)
    
geo_json_data = load_data(
    'https://opendata.arcgis.com/datasets/e7daa4c977d14e1b9e2fa4d7aff81e59_0.geojson',
    'data_json',
     json.load) #json.loads turns geo_json into a dict


In [13]:
#load in income data from csv file
with open('Book2.csv', 'r') as f:
    income = pd.read_csv(f, header=3)

In [14]:
#find the type of desired column
income.dtypes['2015-2019']
income['2015-2019'] = income['2015-2019'].astype(str)
#remove non a-z characters from desired col
income['2015-2019'] = income['2015-2019'].map(lambda x: ''.join([i for i in x if i.isdigit()]))
income['2015-2019'] = pd.to_numeric(income['2015-2019'],errors='coerce')
#remove all NaN values from dataset (unsupported by Choropleth)
income2 = income.dropna(subset=['2015-2019'], axis = 0)
mapping =  dict(zip(income2['Unnamed: 0'].tolist(), income2['2015-2019'].tolist()))
#load in geojson data       
with open('Neighborhood_Statistical_Areas.geojson', 'r') as f:
    data = json.load(f)
for d in geo_json_data["features"]:
    d["GNOCDC_LAB"] = d["properties"]["GNOCDC_LAB"]

In [15]:
proper_name_mapping = {
    "U.S. NAVAL BASE":"U.S. Naval Support Area",
    "ALGIERS POINT":"Algiers Point",
    "WHITNEY":"Whitney",
    "AUDUBON":"Audubon",
    "OLD AURORA":"Old Aurora",
    "B. W. COOPER":"B.W. Cooper",
    "BAYOU ST. JOHN":"Bayou St. John",
    "BEHRMAN":"Behrman",
    "BLACK PEARL":"Black Pearl",
    "BROADMOOR":"Broadmoor",
    "BYWATER":"Bywater",
    "CENTRAL BUSINESS DISTRICT":"Central Business District",
    "CENTRAL CITY":"Central City",
    "CITY PARK":"City Park",
    "DESIRE AREA":"Desire Dev & Neighborhood",
    "DILLARD":"Dillard",
    "DIXON":"Dixon",
    "EAST CARROLLTON":"East Carrollton",
    "EAST RIVERSIDE":"East Riverside",
    "LITTLE WOODS":"Little Woods",
    "FAIRGROUNDS":"Fairgrounds",
    "FILMORE":"Filmore",
    "FISCHER DEV":"Fischer Development",
    "FLORIDA AREA":"Florida Area",
    "FRERET":"Freret",
    "GARDEN DISTRICT":"Garden District",
    "GENTILLY TERRACE":"Gentilly Terrace",
    "GENTILLY WOODS":"Gentilly Woods",
    "GERT TOWN":"Gert Town",
    "HOLLYGROVE":"Hollygrove",
    "HOLY CROSS":"Holy Cross",
    "IBERVILLE":"Iberville Development",
    "IRISH CHANNEL":"Irish Channel",
    "LAKE CATHERINE":"Lake Catherine",
    "LAKE TERRACE & OAKS":"Lake Terrace & Oaks",
    "LAKESHORE - LAKE VISTA":"Lakeshore/Lake Vista",
    "LAKEVIEW":"Lakeview",
    "LAKEWOOD":"Lakewood",
    "WEST END":"West End",
    "LEONIDAS":"Leonidas",
    "LOWER GARDEN DISTRICT":"Lower Garden District",
    "LOWER NINTH WARD":"Lower Ninth Ward",
    "MARIGNY":"Marigny",
    "MARLYVILLE - FONTAINEBLEAU":"Marlyville/Fontainebleau",
    "McDONOGH":"McDonogh",
    "MID-CITY":"Mid-City",
    "MILAN":"Milan",
    "MILNEBURG":"Milneburg",
    "NAVARRE":"Navarre",
    "PINES VILLAGE":"Pines Village",
    "PLUM ORCHARD":"Plum Orchard",
    "PONTCHARTRAIN PARK":"Pontchartrain Park",
    "READ BLVD EAST":"Read Blvd East",
    "NEW AURORA - ENGLISH TURN":"New Aurora/English Turn",
    "SEVENTH WARD":"Seventh Ward",
    "TREME - LAFITTE":"Treme'/Lafitte",
    "ST.  ANTHONY":"St. Anthony",
    "ST. BERNARD AREA":"St. Bernard Area",
    "ST. CLAUDE":"St. Claude",
    "ST. ROCH":"St. Roch",
    "ST. THOMAS DEV":"St. Thomas Development",
    "TALL TIMBERS - BRECHTEL":"Tall Timbers/Brechtel",
    "TOURO":"Touro",
    "TULANE - GRAVIER":"Tulane/Gravier",
    "UPTOWN":"Uptown",
    "VIAVANT - VENETIAN ISLES":"Viavant/Venetian Isles",
    "FRENCH QUARTER":"French Quarter",
    "VILLAGE DE LEST":"Village de l'est",
    "WEST LAKE FOREST":"West Lake Forest",
    "WEST RIVERSIDE":"West Riverside"
}

In [16]:
for d in geo_json_data["features"]:
    if d["GNOCDC_LAB"] not in mapping:
        if d["GNOCDC_LAB"] in proper_name_mapping.keys():
            mapping[d["GNOCDC_LAB"]] = mapping[proper_name_mapping[d["GNOCDC_LAB"]]]
        else:
            mapping[d["GNOCDC_LAB"]] = 0

In [17]:
#https://gis.nola.gov/arcgis/rest/services/Planning/Planning_Districts/MapServer/0?f=pjson
#"nbsphinx": "hidden"
#data loading function
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)
    
planning_districts_json = load_data(
    'https://opendata.arcgis.com/datasets/878cc847e4b6412993c929138333f119_0.geojson',
    'orleans_planning_district.json',
     json.load) #json.loads turns geo_json into a dict
#planning_districts_json

In [37]:
#Make percent units flooded by planning district layer
#load in flooded units data from csv file
with open('Flood_level_planning_district.csv', 'r') as f:
    flood_percent = pd.read_csv(f, header=0)
#flood_percent
mapping2 =  dict(zip(flood_percent['District_Name'].tolist(), flood_percent['Percent_flooded'].tolist()))
#edit json
for f in planning_districts_json["features"]:
    #print the properties of each feature
    #print(f["properties"])
    #create a new variable one level higher in the dictionary
    f["NAME"] = f["properties"]["NAME"]
    #print(f["NAME"])
planning_district_flooding_layer = Choropleth(
                    name = 'Flooding',
                    geo_data=planning_districts_json,
                    choro_data=mapping2,
                    colormap=linear.Blues_05,
                    key_on= 'NAME',
                    style={'opacity': 0.9, 'dashArray': '9', 'fillOpacity': 0.7, 'weight': 1})

In [42]:
# key_on="feature.properties.ID_NIL",
layer = Choropleth(
                    name = 'Income',
                    geo_data=geo_json_data,
                    choro_data=mapping,
                    colormap=linear.Reds_05,
                    #style={'fillOpacity': 1.0, "color":"black"},
                    key_on= 'GNOCDC_LAB',
                    style={'opacity': 0.9, 'dashArray': '9', 'fillOpacity': 0.7, 'weight': 1},
                    hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 1})

m = Map(center=(29.9511, -90.0715), zoom=11)
legend2 = LegendControl({"0-30%":"#f2f2b6", "30-60%":"#eda15f", "60-100%":"#AA5555"}, name="Percent White", position="bottomright")
m.add_control(legend2)
m.add_layer(layer)
legend3 = LegendControl({"0-30%":"#91caff", "30-60%":"#2277f0", "60-90%":"#243ed1"}, name="Units Flooded", position="bottomright")
m.add_control(legend3)
m.add_layer(planning_district_flooding_layer)
control = LayersControl(position='topright')
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(control)
m.layout.height="700px"
m

Map(center=[29.9511, -90.0715], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…